# 1. Parameters, Imports, and Inputs

#### Declare Important Libraries, Parameters for Dataset, and Inputs of the user

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
histogram_bins = 1000
samples = 1000000
training_samples = int(0.9*samples)
testing_samples = int(0.1*samples)

In [2]:
mean = int(input("Please input the mean of the gaussian curve:"))
std_dev = int(input("Please input the stardard deviation of the gaussian curve:"))

Please input the mean of the gaussian curve:0
Please input the stardard deviation of the gaussian curve:1


# 2. Dataset and Histogramming
<ol>
<li> Draw 1,000,00 samples and store it in raw_dataset variable 
<li> Take the minimum and maximum and use linspace to know the starting values of the "bins"
<li> Declare mid_points to take the average of each histogram bins (not the starting points)
<li> Declare count to count how much values are in each "bins"
<li> Start a nested loop that has the histogram bins as the outerloop and random dataset as the inner loop. Take note that this is an inefficient solution since it would loop 1 billion times! See time elapse 
<li> Count the number of occurence and also get the midpoint of each histogram bins
    </ol>

In [3]:
raw_dataset = np.random.normal(mean, std_dev,samples)
starting_points = np.linspace(np.min(raw_dataset), np.max(raw_dataset), num=histogram_bins, endpoint=False)
mid_points = np.zeros(histogram_bins)
count = np.zeros(histogram_bins)

start_time = time.time()
for i in range(histogram_bins):
    #Edge cases
    if i == histogram_bins-1:
        maximum = np.max(raw_dataset)
    else:
        maximum = starting_points[i+1]
    
    #Get the midpoints
    mid_points[i] = starting_points[i]+(starting_points[i] - maximum)/2
    
    #Sweep through the dataset
    for j in raw_dataset:
        if ((j >= starting_points[i]) and (j <= maximum)): #Assume no EXACTLY equal number
            count[i] = count[i] + 1
    
end_time = time.time() - start_time
print("Time took: "+str(end_time))
print("Total number of samples check:"+str(np.sum(count)))

Time took: 231.12993621826172
Total number of samples check:1000000.0


## 2.1 Translate the midpoints and the count to a 1,000,000 random dataset with the midpoints as the value of each histograms.

#### Note that they are only 1,000 values (maximum) as the bins with some other bins near the average having a larger count, hence more number in the dataset.

In [4]:
def translate_count(count, mid_points):
    output = []
    for index, occurrence in enumerate(count):
        if(occurrence !=0):
            for _ in range(int(occurrence)):
                output.append(mid_points[index])
    output_array = np.array(output)
    return output_array

dataset = translate_count(count,mid_points)
random_dataset = np.random.choice(dataset,samples,replace=False)
training_dataset = random_dataset[0:training_samples]
testing_dataset = random_dataset[training_samples:(training_samples + testing_samples)]

print("Size of testing dataset (histogrammed):"+str(training_dataset.shape))
print("Size of testing dataset (histogrammed):"+str(testing_dataset.shape))

Size of testing dataset (histogrammed):(900000,)
Size of testing dataset (histogrammed):(100000,)


<h1> 3. Build a 3 layer neural network </h1>
    
Strong inspiration: https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

In [5]:
nn_architecture = [
    {"input_dim": 1, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 1, "activation": "relu"},
]

def init_layers(nn_architecture, seed = 99):
    #np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.normal(0,0.1,(layer_output_size, layer_input_size))
        params_values['b' + str(layer_idx)] = np.random.normal(0,0.1,(layer_output_size, 1))
        
    return params_values


def relu(Z):
    return np.maximum(0,Z)
    
def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;
    


def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation == "relu":
        activation_func = relu
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

def get_cost_value(predictions,targets):
    # Retrieving number of samples in dataset
    samples_num = len(predictions)
    
    # Summing square differences between predicted and expected values
    accumulated_error = 0.0
    for prediction, target in zip(predictions, targets):
        accumulated_error += (prediction - target)**2
        
    # Calculating mean and dividing by 2
    mae_error = (1.0 / (2*samples_num)) * accumulated_error
    
    return mae_error

def get_accuracy_value(Y_hat, Y):
    return 100 - np.mean(np.abs(Y_hat - Y)) * 100

def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation == "relu":
        backward_activation_func = relu_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    
    dA_prev = (Y_hat - Y)
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
        
    
    return grads_values

def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx+1)] -= learning_rate * grads_values["dW" + str(layer_idx+1)]        
        params_values["b" + str(layer_idx+1)] -= learning_rate * grads_values["db" + str(layer_idx+1)]

    return params_values;

def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 1)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        for j in range(X.shape[1]):
            Y_hat, cashe = full_forward_propagation(np.array([X[0][j]])[np.newaxis,:], params_values, nn_architecture)
            cost = get_cost_value(Y_hat, np.array([Y[0][j]])[np.newaxis,:])
            cost_history.append(cost)
            accuracy = get_accuracy_value(Y_hat, np.array([Y[0][j]])[np.newaxis,:])
            accuracy_history.append(accuracy)
        
            grads_values = full_backward_propagation(Y_hat, np.array([Y[0][j]])[np.newaxis,:], cashe, params_values, nn_architecture)
        
            params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values, cost_history, accuracy_history 
        
def test(X, Y, nn_architecture,params_values):
    Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
    cost = get_cost_value(Y_hat, Y)
    accuracy = get_accuracy_value(Y_hat, Y)

    return accuracy

def normal(x,mu,sigma):
    return ( 2.*np.pi*sigma**2. )**-.5 * np.exp( -.5 * (x-mu)**2. / sigma**2. )

def real_function(inputs,mean,std_dev):
    output = []
    for i in range(inputs.shape[0]):
        gaussian_eq = normal(inputs[i],mean,std_dev)
        output.append(relu(gaussian_eq))
    
    return np.array(output)[np.newaxis,:]


In [6]:
y = real_function(training_dataset,mean,std_dev)
params_values, cost_history, accuracy_history = train(training_dataset[np.newaxis,:],y,nn_architecture,1,0.1)
print(accuracy_history)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
z = real_function(testing_dataset,mean,std_dev)
print(test(testing_dataset[np.newaxis,:],z,nn_architecture,params_values))
print(test(testing_dataset[np.newaxis,:],z,nn_architecture,params_values))

hi = training_dataset[np.newaxis,:]

np.array([hi[0][1]])[np.newaxis,:]

#np.array([training_dataset[0]])[np.newaxis,:]

71.80346654591341
71.80346654591341


array([[1.48787931]])